In [1]:
from utils import HParam
hp = HParam('config/finetune_baichuan.yaml')
hp.llama.train_batchsize = hp.data.train_batchsize


In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("baichuan-inc/Baichuan-7B",
                                          cache_dir = hp.llama.cache_dir, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained("baichuan-inc/Baichuan-7B", 
                                             device_map="auto", trust_remote_code=True,
                                             cache_dir = hp.llama.cache_dir)



[2023-07-26 11:43:15,825] [INFO] [real_accelerator.py:133:get_accelerator] Setting ds_accelerator to cuda (auto detect)


In [7]:
inputs = tokenizer('14时23分;山地地形;敌方坦克出现在侧面战场;我方直升机进入维修状态;', return_tensors='pt')
inputs = inputs.to('cuda:0')
pred = model_lora.generate(**inputs, max_new_tokens=256,repetition_penalty=1.1)
print(tokenizer.decode(pred.cpu()[0], skip_special_tokens=True))

14时23分;山地地形;敌方坦克出现在侧面战场;我方直升机进入维修状态;
5.6秒后,敌人开始攻击。此时,在地图上显示的“战斗”区域内有两个目标:一个是在右边的小型建筑物中被击中的小型车辆(红色),另一个是位于左边的重型装甲车(绿色)。由于该地区没有其他障碍物或掩体,因此可以很容易地看到它们之间的距离和位置关系。此外,还可以通过观察周围环境来判断其移动方向以及可能采取的行动方式等信息。


In [ ]:
from llama_model.llama import LlamaPeftGenerate

model_lora = LlamaPeftGenerate(hp.llama,hp.lora)

In [8]:
from llama_model import LlamaModule
gpt2_litmodel = LlamaModule
print('load from checkpoint')
gpt2_litmodel = gpt2_litmodel.load_from_checkpoint('/root/autodl-tmp/transfer/20230721_4a100_baichuan/epoch=epoch=0069-validation.loss=val_loss=5.296.ckpt', 
                                                   args=hp.llama, model=model_lora,tokenizer=tokenizer)

load from checkpoint


OutOfMemoryError: CUDA out of memory. Tried to allocate 172.00 MiB (GPU 0; 47.37 GiB total capacity; 46.17 GiB already allocated; 53.19 MiB free; 46.18 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
pred = gpt2_litmodel.model.generate(**inputs, max_new_tokens=256,repetition_penalty=1.1)
print(tokenizer.decode(pred.cpu()[0], skip_special_tokens=True))